**SA433 &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2025**

# Lesson 18. Tidy Data

## In this lesson...

- We'll learn about **tidy data**: a way to consistently organize tabular data

- We'll also learn some techniques on how to make data tidy
    - These techniques are useful in general, too

- The concept of tidy data was originally proposed by [Hadley Wickham](http://hadley.nz/), Chief Scientist at RStudio

- Many statistical and visualization packages in Python (and R) are designed to work with tidy data, like Altair!

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## A few example datasets 

* Let's start by importing Pandas and NumPy:

In [1]:
import pandas as pd
import numpy as np

- We'll use the following datasets in this lesson:

In [2]:
table1 = pd.read_csv('data/table1.csv')
table2 = pd.read_csv('data/table2.csv')
table3 = pd.read_csv('data/table3.csv')
table4a = pd.read_csv('data/table4a.csv')
table4b = pd.read_csv('data/table4b.csv')

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## What is tidy data?

- We can represent the same underlying data in multiple ways


- Throughout this course so far, we've used the terms *columns* and *variables* interchangeably, as well as the terms *rows* and *observations*


- However, depending on how the data is organized and the information it contains, this may not be correct


- Below, we have 4 representations of the same data: `table1`, `table2`, `table3` and `table4a` + `table4b`


- In this data, we have four variables: `country`, `year`, `population`, and `cases`


- Each observation corresponds to a `country`-`year` pair


- Each of the 4 representations shows the same values, but organized differently:

In [3]:
table1

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [4]:
table2

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


In [5]:
table3

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


In [6]:
# Together with table4b below
table4a

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


In [7]:
# Together with table4a above
table4b

,country,1999,2000
0,Afghanistan,19987071,20595360
1,Brazil,172006362,174504898
2,China,1272915272,1280428583


- A dataset is **tidy** if:
    1. Each variable has its own column
    2. Each observation has its own row
    3. Each value has its own cell

**Question.** Which of the four representations of the dataset above are tidy?

*Write your notes here. Double-click to edit.*

*Solution.* Only `table1` above is tidy. It's the only representation where each column is a variable.

- The principles of tidy data seem obvious, but most data that we encounter in the wild is *not* tidy 


- Given a dataset, we need to first figure out what the variables and observations are; then we can make it tidy


- Next, we'll learn a few techniques that can help make a dataset tidy

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Pivoting from wide to long form

- A common problem is a dataset where some of the column names are not the names of variables, but the *values* of a variable


- For example, let's look at `table4a` again:

In [8]:
table4a

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


- In this dataset:
    - the column names `1999` and `2000` represent values of the `year` variable
    - the values in these columns represent values of the `cases` variable
    - each row represents 2 observations, not 1

- To tidy a dataset like this, we need to **pivot** the offending columns into a new pair of variables


- We can accomplish this with the `.melt()` DataFrame method

- We need 3 parameters:

    1. `id_vars`: a list of columns to keep as-is
        - The other columns represent data that will be moved to the column created by `var_name`
    2. `var_name`: the name of the column to create from the data stored in the column names
    3. `value_name`: the name of the column to create from the data stored in the column values

- Visually:

<img src='img/melt.jpg' width=700 />


- For example, we can pivot the columns `1999` and `2000` in `table4a` into new variables called `year` and `cases`, like this:

In [9]:
# Solution
(
    table4a
    .melt(
        id_vars=['country'], 
        var_name='year', 
        value_name='cases'
    )
)

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


- This is often called **pivoting from wide to long form** because it makes datasets "longer" by increasing the number of rows and decreasing the number of columns

- We can use `.melt()` in a similar fashion to tidy `table4b`, which contains the value of the variable `population`:

In [10]:
# Solution
(
    table4b
    .melt(
        id_vars=['country'], 
        var_name='year', 
        value_name='population'
    )
)

,country,year,population
0,Afghanistan,1999,19987071
1,Brazil,1999,172006362
2,China,1999,1272915272
3,Afghanistan,2000,20595360
4,Brazil,2000,174504898
5,China,2000,1280428583


- We'll learn how to merge these `melt`ed DataFrames into a single DataFrame in a future lesson

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Pivoting from long to wide form

- Another common problem is a dataset with each observation scattered across multiple rows


- For example, consider `table2`: each observation corresponds to a `country`-`year` pair, but each observation is spread across 2 rows

In [11]:
table2

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


- To tidy this up, we can use the `.pivot_table()` method

- We need 3 parameters:
    1. `index`: the variables that identify a single observation
    2. `columns`: the column to take variable names from
    3. `values`: the column to take values from

- Visually:

<img src='img/pivot_table.jpg' width=800 />


- For example, we can pivot the `type` and `count` columns of `table2` like this:

In [12]:
# Solution
(
    table2
    .pivot_table(
        index=['country', 'year'],
        columns='type',
        values='count'
    )
    .reset_index()
    .rename_axis(columns=None)
)

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


- `.reset_index()` converts the existing index into ordinary columns, and resets the index of the DataFrame to the default one (consecutive integers)


- `.rename_axis(columns=None)` removes the name of the column axis generated by `.pivot_table()`


- These steps are often desirable when performing additional wrangling or analysis steps

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Separating

- Let's take a look at `table3`:

In [13]:
table3

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


- To tidy this data, we need to split the contents of `rate` into two columns, `cases` and `population`

- We can accomplish this with the `.str.split()` Series method, with the following keyword arguments:
    - `pat=...` specifies a string to use as a separator
        - If `pat` is not specified, the method will split on whitespace
    - `expand=True` tells the method to output split strings into multiple columns/Series in a DataFrame

- So, we can split the contents of `rate` in `table3` like this:

In [14]:
# Solution
table3_cases_pop = table3['rate'].str.split(pat='/', expand=True)
table3_cases_pop

,0,1
0,745,19987071
1,2666,20595360
2,37737,172006362
3,80488,174504898
4,212258,1272915272
5,213766,1280428583


- Note that after splitting `rate`, we still have strings instead of numeric values as output:

In [15]:
table3_cases_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6 non-null      object
 1   1       6 non-null      object
dtypes: object(2)
memory usage: 228.0+ bytes


- We can add the split contents of `rate` to `table3`, convert them to integers, and drop `rate` from the table, like this:

In [16]:
# Solution
(
    table3
    .assign(
        cases=lambda x: x['rate'].str.split(pat='/', expand=True)[0].astype(int),
        population=lambda x: x['rate'].str.split(pat='/', expand=True)[1].astype(int)
    )
    .drop(columns=['rate'])
)

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


- If the data you want to separate doesn't contain a separator character, you can use Python slicing notation with `.str` to take substrings of the data


- For example, we can split `year` into `century` and `year` like this:

In [17]:
# Solution
table1_sep = table1.assign(
    century=lambda x: x['year'].astype(str).str[0:2],
    year=lambda x: x['year'].astype(str).str[2:4]
)

table1_sep

,country,year,cases,population,century
0,Afghanistan,99,745,19987071,19
1,Afghanistan,00,2666,20595360,20
2,Brazil,99,37737,172006362,19
3,Brazil,00,80488,174504898,20
4,China,99,212258,1272915272,19
5,China,00,213766,1280428583,20


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Uniting

- The `.str.cat()` Series method is used to concatenate strings
    - `sep=...` specifies the separator to use between the strings 
        - By default, the separator is the empty string `''`

- For example, we can reverse the split we performed above and join `century` and `year` back together, like this:

In [18]:
# Solution
(
    table1_sep
    .assign(
        year=lambda x: x['century'].str.cat(x['year']).astype(int)
    )
    .drop(columns=['century'])
)

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

### Problem 0

In the same folder as this notebook, there is a CSV file `data/who.csv`, which contains a subset of data from the World Health Organization Global Tuberculosis Report.

The problems below will walk you through tidying this data.

First, read the CSV file into a DataFrame called `who`. Use `.head()` and `.info()` to get a sense of the data.

In [19]:
# Solution
who = pd.read_csv('data/who.csv')

In [20]:
# Solution
who.head()

,country,iso2,iso3,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,...,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
0,Afghanistan,AF,AFG,1980,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AF,AFG,1981,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AF,AFG,1982,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AF,AFG,1983,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AF,AFG,1984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Solution
who.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7240 entries, 0 to 7239
Data columns (total 60 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       7240 non-null   object 
 1   iso2          7206 non-null   object 
 2   iso3          7240 non-null   object 
 3   year          7240 non-null   int64  
 4   new_sp_m014   3173 non-null   float64
 5   new_sp_m1524  3209 non-null   float64
 6   new_sp_m2534  3206 non-null   float64
 7   new_sp_m3544  3219 non-null   float64
 8   new_sp_m4554  3223 non-null   float64
 9   new_sp_m5564  3218 non-null   float64
 10  new_sp_m65    3209 non-null   float64
 11  new_sp_f014   3174 non-null   float64
 12  new_sp_f1524  3194 non-null   float64
 13  new_sp_f2534  3200 non-null   float64
 14  new_sp_f3544  3199 non-null   float64
 15  new_sp_f4554  3204 non-null   float64
 16  new_sp_f5564  3195 non-null   float64
 17  new_sp_f65    3197 non-null   float64
 18  new_sn_m014   1045 non-null 

### Problem 1

The columns `country`, `iso2`, and `iso3` redundantly specify the country. Verify this by grouping the data by `country` and counting the number of unique values of `iso2` and `iso3` for each group. *Hint.* Use the `nunique` reduction/aggregation method.

In [22]:
# Solution
(
    who
    .groupby(['country'])
    .agg(
        iso2_nunique=('iso2', 'nunique'),
        iso3_nunique=('iso3', 'nunique')
    )
)

,iso2_nunique,iso3_nunique
country,,
Afghanistan,1,1
Albania,1,1
Algeria,1,1
American Samoa,1,1
Andorra,1,1
...,...,...
Wallis and Futuna Islands,1,1
West Bank and Gaza Strip,1,1
Yemen,1,1


### Problem 2

Now that you've established that `iso2` and `iso3` are not needed, create a DataFrame `who1` without those redundant columns.

In [23]:
# Solution
who1 = (
    who
    .drop(columns=['iso2', 'iso3'])
)

who1.head()

,country,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,new_sp_m65,new_sp_f014,...,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
0,Afghanistan,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Problem 3

The columns from `new_sp_m014` to `newrel_f65` specify the number of cases in each country-year for different situations.

Pivot the columns in `who1` from `new_sp_m014` to `newrel_f65` into a new variable called `key` (a generic name for now). Drop all rows in the resulting DataFrame with NA values. Put the result in a new DataFrame called `who2`.

In [24]:
# Solution
who2 = (
    who1
    .melt(
        id_vars=['country', 'year'],
        var_name='key', 
        value_name='cases'
    )
    .dropna()
)

who2.tail()

,country,year,key,cases
405269,Viet Nam,2013,newrel_f65,3110.0
405303,Wallis and Futuna Islands,2013,newrel_f65,2.0
405371,Yemen,2013,newrel_f65,360.0
405405,Zambia,2013,newrel_f65,669.0
405439,Zimbabwe,2013,newrel_f65,725.0


### Problem 4

Next, let's parse the values in the `key` column of `who2`:

- The first 3 letters denote whether the observation represents new or old cases of TB
    - Note that this dataset contains only new cases

- The next 2-3 letters describe the type of TB:
    - `rel` = cases of relapse
    - `ep` = cases of extrapulmonary TB
    - `sn` = cases of pulmonary TB that could not be diagnosed by a pulmonary smear (smear negative)
    - `sp` = cases of pulmonary TB that could be diagnosed by a pulmonary smear (smear positive)

- The 6th letter gives the sex of TB patients: `m` for male, `f` for female    

- The remaining number gives the age group:
    - `014` = 0-14 years old
    - `1524` = 15-24 years old
    - `2534` = 25-34 years old
    - `3544` = 35-44 years old
    - `4554` = 45-54 years old
    - `5564` = 55-64 years old
    - `65` = 65 or older

Note that the `key` values are slightly inconsistent: most use `_` to separate the first 3 letters from the type of TB, except those that have `newrel` instead of `new_rel`.

Create a new DataFrame called `who3`, that replaces the values of `newrel` with `new_rel` in the `key` column of `who2`. 

*Hint.* Use the `.str.replace()` method. [Here's the documentation.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html)

In [25]:
# Solution
who3 = (
    who2
    .assign(
        key=lambda x: x['key'].str.replace('newrel', 'new_rel')
    )
)

who3.tail()

,country,year,key,cases
405269,Viet Nam,2013,new_rel_f65,3110.0
405303,Wallis and Futuna Islands,2013,new_rel_f65,2.0
405371,Yemen,2013,new_rel_f65,360.0
405405,Zambia,2013,new_rel_f65,669.0
405439,Zimbabwe,2013,new_rel_f65,725.0


### Problem 5

Now that the values in `key` are consistently separated by `_`, split `key` into three columns: `new`, `type`, `sexage`. Drop the `key` column, and put the results into a DataFrame called `who4`.

In [26]:
# Solution
who4 = (
    who3
    .assign(
        new=lambda x: x['key'].str.split(pat='_', expand=True)[0],
        type=lambda x: x['key'].str.split(pat='_', expand=True)[1],
        sexage=lambda x: x['key'].str.split(pat='_', expand=True)[2]
    )
    .drop(columns=['key'])
)

who4.head()

,country,year,cases,new,type,sexage
17,Afghanistan,1997,0.0,new,sp,m014
18,Afghanistan,1998,30.0,new,sp,m014
19,Afghanistan,1999,8.0,new,sp,m014
20,Afghanistan,2000,52.0,new,sp,m014
21,Afghanistan,2001,129.0,new,sp,m014


### Problem 6

Split the column `sexage` into 2 columns: `sex` and `age`. Note that the first character of `sexage` is always either `m` or `f`. Drop the `sexage` column, and put the results into a DataFrame called `who5`.

In [27]:
# Solution
who5 = (
    who4
    .assign(
        sex=lambda x: x['sexage'].str[0],
        age=lambda x: x['sexage'].str[1:]
    )
    .drop(columns=['sexage'])
)

who5.head()

,country,year,cases,new,type,sex,age
17,Afghanistan,1997,0.0,new,sp,m,014
18,Afghanistan,1998,30.0,new,sp,m,014
19,Afghanistan,1999,8.0,new,sp,m,014
20,Afghanistan,2000,52.0,new,sp,m,014
21,Afghanistan,2001,129.0,new,sp,m,014


### Problem 7 

Now the dataset is tidy! Put together your code from Problems 2-6 into a single "tidyfying" method chain. Merge method calls where appropriate.

In [28]:
# Solution
(
    who
    .drop(columns=['iso2', 'iso3'])
    .melt(
        id_vars=['country', 'year'],
        var_name='key', 
        value_name='cases'
    )
    .dropna()
    .assign(
        key=lambda x: x['key'].str.replace('newrel', 'new_rel'),
        new=lambda x: x['key'].str.split(pat='_', expand=True)[0],
        type=lambda x: x['key'].str.split(pat='_', expand=True)[1],
        sexage=lambda x: x['key'].str.split(pat='_', expand=True)[2],
        sex=lambda x: x['sexage'].str[0],
        age=lambda x: x['sexage'].str[1:]
    )
    .drop(columns=['key', 'sexage'])
)

,country,year,cases,new,type,sex,age
17,Afghanistan,1997,0.0,new,sp,m,014
18,Afghanistan,1998,30.0,new,sp,m,014
19,Afghanistan,1999,8.0,new,sp,m,014
20,Afghanistan,2000,52.0,new,sp,m,014
21,Afghanistan,2001,129.0,new,sp,m,014
...,...,...,...,...,...,...,...
405269,Viet Nam,2013,3110.0,new,rel,f,65
405303,Wallis and Futuna Islands,2013,2.0,new,rel,f,65
405371,Yemen,2013,360.0,new,rel,f,65
405405,Zambia,2013,669.0,new,rel,f,65


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)

- Lesson and problems inspired by Chapter 12 of [R for Data Science](https://r4ds.had.co.nz/)    